# Ensemble Methods

## Ensemble Method 1: Majority voting

In [ ]:
#nsemble method to try is regular majority vote
import pickle
import pandas as pd
import numpy as np
import operator
from scipy import stats
import cellpylib as cpl
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec



----------------------

Loading the traing set here itself for easy access

------------------------

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
data = pd.read_csv('weatherdatafrom12-19_with_fire_v1.csv')
cols=['TEMP','DEWP','SLP','STP','VISIB','WDSP','MXSPD','GUST','MAX','MIN','PRCP']
X = data[cols]
y=data['Class']
from sklearn import preprocessing

# Imputation of missing values
X = X.fillna(X.mean())

X_scaled = preprocessing.scale(X)
X_scaled.mean(axis=0)
X_scaled.std(axis=0)
scaler = preprocessing.StandardScaler().fit(X)
scaler.mean_
scaler.scale_ 
X = scaler.transform(X)
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [ ]:
#from keras.optimizers import Adam
from tensorflow import keras
logistic_model = pickle.load(open("Weather_SVM.pickle", 'rb'))
random_forest_model = pickle.load(open("Weather_RandomForest.pickle", 'rb'))
from tensorflow import keras
cnn_model = keras.models.load_model('NDVI_CNN.h5')
'''model = keras.models.load_model('NDVI_CNN.model.h5', custom_objects={
    'Adam': lambda **kwargs: hvd.DistributedOptimizer(keras.optimizers.Adam(learning_rate=0.001))
})'''

#nb_model= pickle.load(open("../NB_Model_pickle", 'rb'))

#dtree_model= pickle.load(open("../DTree_model_pickle", 'rb'))

In [ ]:
#X = np.load('../test_data/test.np.npy')
#Y = np.load('../test_data/test_labels.np.npy')
X = X_test
Y = y_test



In [ ]:
log_pred = logistic_model.predict(X)
print(log_pred)

In [ ]:
rf_pred = random_forest_model.predict(X)
print(rf_pred)

In [ ]:
comb = np.vstack((log_pred, rf_pred ) )
#comb = np.vstack((log_pred, rf_pred, knn_pred, dtree_pred) )
vote = stats.mode(comb, axis=0)
Y_pred = vote[0][0]

c = np.logical_xor(Y_pred, Y.T[0])
incorrect =np.sum(c)
accuracy = 1 - (incorrect/Y.shape[0])
print(accuracy)


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_matx = confusion_matrix(y_test, Y_pred)
sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
plt.show()

## Ensemble Method 2: Majority voting using cellular automata

In [ ]:
# Trying ensemble method 1 - Apply rule 232
# Rule = Mod[p q + p r + q r, 2]

#cellular_automaton = cpl.init_random(5)
#print(cellular_automaton)
ca_list = []
titles = []
i= 1
for column in comb.T:
   
    cellular_automaton = np.array([column])
    
     #This line will be the actual classifier output

    c = mpl.colors.ListedColormap(['green', 'red', 'black', 'gray'])
    n = mpl.colors.Normalize(vmin=0,vmax=3)   


    # evolve the CA for 100 time steps, using Rule 232 as defined in NKS
    cellular_automaton = cpl.evolve(cellular_automaton, timesteps=100, 
                                apply_rule=lambda n, c, t: cpl.nks_rule(n, 232))
    titles.append("Vote for " + str(i))
    i+=1
    ca_list.append(cellular_automaton)
    
    

In [ ]:
#votes = [ca_list.apply(lambda x : np.argmax(np.bincount(x[49])))]
votes = map(lambda x : np.argmax(np.bincount(x[99])), ca_list)
Y_pred = np.array(list(votes))
print(Y_pred)

In [ ]:
c = np.logical_xor(Y_pred, Y.T[0])
incorrect =np.sum(c)
accuracy = 1 - (incorrect/Y.shape[0])
print(accuracy)

In [ ]:
#cpl.plot_multiple(ca_list, titles)
fig = plt.figure(figsize=(30,10)) 

gs = gridspec.GridSpec(cellular_automaton.shape[0], cellular_automaton.shape[1])

maxj=cellular_automaton.shape[0]
maxk = cellular_automaton.shape[1]

j=0
k=0
for i in ca_list:
    
    
    ax = plt.subplot(gs[j, k])
    ax.set_ylabel('time') 
    fig.add_subplot(ax) 
    if k >= maxk-1 :
        k = 0
        if j >= maxj-1:
            j +=  0
        else:
            j+=1
    else:
        k +=1
    
        
       
plt.show()

## Ensemble Method 3 : Classificational Cellular automata

`Inference Algorithm:
    Input: a sample for classification 
    Number of iterations: t=1,2,...V 
For t=1,2...V 
    -each cell in automaton classifies the sample
    -change cells energy according to the transaction rules 
    -each cell with energy below zero does not survive
Classify the sample according to the weighted voting of the survived cells 

Output: class of the input sample  
    
Transaction rules:
    score = support * distance
    

(4) (PDF) Machine-Learning with Cellular Automata. Available from: https://www.researchgate.net/publication/221460875_Machine-Learning_with_Cellular_Automata [accessed May 06 2019].`

In [ ]:
ind = np.random.randint(X.shape[0], size=1)
print(ind)
X_test = X[ind, :]
print(Y[ind])

In [ ]:
models_list = [logistic_model, random_forest_model]


preds_list = []
for model in models_list:
    preds_list.append(model.predict(X_test))
    
print(np.asarray(preds_list).flatten())
np.asarray(preds_list)

In [ ]:
preds_list